In [24]:
# -*- coding: cp936 -*-
import cv2
import numpy as np
import glob

# Ref: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

# 设置寻找亚像素角点的参数，采用的停止准则是最大循环次数30和最大误差容限0.001
# # termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.001)

# 获取标定板角点的位置
objp = np.zeros((4*6,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:4].T.reshape(-1,2)  # 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y

obj_points = []  # 3d point in real world space
img_points = []  # 2d points in image plane.

images = glob.glob("./2023*.jpg")
i=0;
for fname in images:
    print (f'reading {fname}')
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    size = gray.shape[::-1]
    ret, corners = cv2.findChessboardCorners(gray, (6, 4), None)
    #print(corners)

    # If found, add object points, image points (after refining them)
    if ret:

        obj_points.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)  # 在原角点的基础上寻找亚像素角点
        #print(corners2)
        if [corners2]:
            img_points.append(corners2)
        else:
            img_points.append(corners)

        cv2.drawChessboardCorners(img, (6, 4), corners, ret)  
        i+=1;
        cv2.imwrite('conimg'+str(i)+'.jpg', img)
        cv2.waitKey(30)

print(len(img_points))
cv2.destroyAllWindows()

# Calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, size, None, None)

print("ret:", ret)
print("mtx:\n", mtx) # 内参数矩阵
print("dist:\n", dist)  # 畸变系数   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)
print("rvecs:\n", rvecs)  # 旋转向量  # 外参数
print("tvecs:\n", tvecs ) # 平移向量  # 外参数

print("-----------------------------------------------------")

reading .\2023-04-11-16-55-39_0.jpg
reading .\2023-04-11-16-55-41_1.jpg
reading .\2023-04-11-16-55-42_2.jpg
reading .\2023-04-11-16-55-43_3.jpg
reading .\2023-04-11-16-55-44_4.jpg
reading .\2023-04-11-16-55-54_5.jpg
reading .\2023-04-11-16-55-55_6.jpg
reading .\2023-04-11-16-55-57_7.jpg
reading .\2023-04-11-16-55-59_8.jpg
reading .\2023-04-11-16-56-05_9.jpg
reading .\2023-04-11-16-56-06_10.jpg
reading .\2023-04-11-16-56-07_11.jpg
reading .\2023-04-11-16-56-09_12.jpg
reading .\2023-04-11-16-56-10_13.jpg
reading .\2023-04-11-16-56-14_14.jpg
reading .\2023-04-11-16-56-15_15.jpg
16
ret: 0.15502244554890912
mtx:
 [[410.58106697   0.         308.72783244]
 [  0.         410.27753768 232.6058636 ]
 [  0.           0.           1.        ]]
dist:
 [[ 0.06861641 -0.15402306  0.00017448 -0.00226764  0.13321252]]
rvecs:
 (array([[0.02541369],
       [0.09159436],
       [0.16779176]]), array([[-0.05160414],
       [ 0.0953814 ],
       [ 0.16053818]]), array([[-0.10722726],
       [ 0.09035805],


In [34]:
img = cv2.imread(images[2])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))#显示更大范围的图片（正常重映射之后会删掉一部分图像）
print (f'newcameramtx:\n{newcameramtx}')
print("------------------使用undistort函数-------------------")
dst = cv2.undistort(img,mtx,dist,None,newcameramtx)
x,y,w,h = roi
dst1 = dst[y:y+h,x:x+w]
print (f'roi:\n{roi}')
cv2.imwrite('calibresult3.jpg', dst1)
print ("方法一:dst的大小为:", dst1.shape)

newcameramtx:
[[415.71887207   0.         306.98727511]
 [  0.         413.60134888 232.24137831]
 [  0.           0.           1.        ]]
------------------使用undistort函数-------------------
roi:
(5, 5, 626, 468)
方法一:dst的大小为: (468, 626, 3)


In [28]:
print (type(tvecs))
for t in tvecs:
    print (t, end='\n@@@@@@@@@@@@@\n')

<class 'tuple'>
[[-2.89206321]
 [-1.82887412]
 [ 8.80780309]]
@@@@@@@@@@@@@
[[-2.81691139]
 [-1.07971572]
 [ 8.9007603 ]]
@@@@@@@@@@@@@
[[-2.92454922]
 [-0.50603714]
 [ 8.84124413]]
@@@@@@@@@@@@@
[[-3.30297806]
 [-0.97282997]
 [ 8.67307458]]
@@@@@@@@@@@@@
[[-4.1228464 ]
 [-0.56644984]
 [ 8.28918308]]
@@@@@@@@@@@@@
[[ 1.64806345]
 [-2.48529198]
 [10.32080125]]
@@@@@@@@@@@@@
[[ 1.98227405]
 [-2.11712317]
 [ 9.79471403]]
@@@@@@@@@@@@@
[[ 1.94398265]
 [-1.53093786]
 [ 9.81750784]]
@@@@@@@@@@@@@
[[ 2.2237651 ]
 [-1.35305955]
 [ 9.8847671 ]]
@@@@@@@@@@@@@
[[ 1.73911328]
 [-3.98712883]
 [ 9.87680715]]
@@@@@@@@@@@@@
[[ 2.71637027]
 [-3.42859609]
 [10.12246057]]
@@@@@@@@@@@@@
[[ 3.21017642]
 [-3.42486081]
 [10.14886451]]
@@@@@@@@@@@@@
[[ 2.77129317]
 [-4.52854914]
 [ 9.71221304]]
@@@@@@@@@@@@@
[[ 3.03980558]
 [-3.95403572]
 [10.08076389]]
@@@@@@@@@@@@@
[[-2.16541494]
 [-3.12682628]
 [10.67887599]]
@@@@@@@@@@@@@
[[-2.6685977 ]
 [-3.55030656]
 [10.39899589]]
@@@@@@@@@@@@@


In [31]:
print (len(rvecs))
for r in rvecs:
    print (r, end='\n@@@@@@@@@@@@@\n')

16
[[0.02541369]
 [0.09159436]
 [0.16779176]]
@@@@@@@@@@@@@
[[-0.05160414]
 [ 0.0953814 ]
 [ 0.16053818]]
@@@@@@@@@@@@@
[[-0.10722726]
 [ 0.09035805]
 [ 0.16007893]]
@@@@@@@@@@@@@
[[-0.04591842]
 [ 0.04896138]
 [ 0.13079415]]
@@@@@@@@@@@@@
[[-0.08512043]
 [-0.02994789]
 [ 0.09729993]]
@@@@@@@@@@@@@
[[-0.06741072]
 [-0.04004056]
 [ 1.4661484 ]]
@@@@@@@@@@@@@
[[-0.09733467]
 [-0.01779252]
 [ 1.50057371]]
@@@@@@@@@@@@@
[[-0.16518863]
 [-0.00564375]
 [ 1.48839749]]
@@@@@@@@@@@@@
[[-0.17939904]
 [ 0.00728736]
 [ 1.50853297]]
@@@@@@@@@@@@@
[[ 0.00245681]
 [-0.09221711]
 [ 1.46451163]]
@@@@@@@@@@@@@
[[0.01861198]
 [0.00238902]
 [1.48173392]]
@@@@@@@@@@@@@
[[0.05457253]
 [0.03516414]
 [1.45852539]]
@@@@@@@@@@@@@
[[ 0.12067205]
 [-0.03136977]
 [ 1.38826205]]
@@@@@@@@@@@@@
[[0.14848275]
 [0.11244353]
 [1.14337677]]
@@@@@@@@@@@@@
[[0.02582055]
 [0.09711904]
 [0.03082211]]
@@@@@@@@@@@@@
[[0.06492623]
 [0.0613653 ]
 [0.0376975 ]]
@@@@@@@@@@@@@
